In [1]:
import pandas as pd  
import pickle        

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import LabelEncoder


In [7]:
df=pd.read_csv('loan_prediction.csv')
df.head()

,Unnamed: 0,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
2,3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
3,4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
4,5,21.0,female,High School,12951.0,0,OWN,2500.0,VENTURE,7.14,0.19,2.0,532,No,1


In [8]:
df=df.drop(['Unnamed: 0','person_education','person_gender','loan_percent_income'],axis=1)
df.head()

,person_age,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,71948.0,0,RENT,35000.0,PERSONAL,16.02,3.0,561,No,1
1,25.0,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,3.0,635,No,1
2,23.0,79753.0,0,RENT,35000.0,MEDICAL,15.23,2.0,675,No,1
3,24.0,66135.0,1,RENT,35000.0,MEDICAL,14.27,4.0,586,No,1
4,21.0,12951.0,0,OWN,2500.0,VENTURE,7.14,2.0,532,No,1


In [9]:
df['loan_status'].value_counts()

loan_status
0    12000
1    10000
Name: count, dtype: int64

In [10]:
df=df[df['person_age']<100]
df.describe()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,cb_person_cred_hist_length,credit_score,loan_status
count,21997.000000,2.199700e+04,21997.000000,21997.000000,21997.000000,21997.000000,21997.000000,21997.000000
mean,29.258490,7.512687e+04,6.733327,9952.679775,11.549953,6.899714,633.827386,0.454607
std,7.124086,6.918483e+04,7.175773,6561.095247,3.026160,4.587666,50.857061,0.497947
min,20.000000,8.000000e+03,0.000000,563.000000,5.420000,2.000000,418.000000,0.000000
25%,24.000000,4.286500e+04,1.000000,5000.000000,9.450000,3.000000,602.000000,0.000000
50%,27.000000,6.213000e+04,4.000000,8400.000000,11.230000,5.000000,640.000000,0.000000
75%,33.000000,8.965500e+04,10.000000,13650.000000,13.640000,9.000000,671.000000,1.000000
max,94.000000,2.448661e+06,76.000000,35000.000000,20.000000,30.000000,784.000000,1.000000


In [11]:
df['loan_status']=[i if i>700 else 0 for i in df['credit_score']] 
df['loan_status'].value_counts()
                   

loan_status
0      20499
702       90
704       84
703       77
701       73
       ...  
773        1
765        1
772        1
768        1
767        1
Name: count, Length: 64, dtype: int64

In [12]:
copied=df.copy()
copied.to_csv('cleaned.csv')
df.columns

Index(['person_age', 'person_income', 'person_emp_exp',
       'person_home_ownership', 'loan_amnt', 'loan_intent', 'loan_int_rate',
       'cb_person_cred_hist_length', 'credit_score',
       'previous_loan_defaults_on_file', 'loan_status'],
      dtype='object')

In [13]:
obj=LabelEncoder()
for i in df.columns:
    if df[i].dtype==object:
        df[i]=obj.fit_transform(df[i])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21997 entries, 0 to 21999
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      21997 non-null  float64
 1   person_income                   21997 non-null  float64
 2   person_emp_exp                  21997 non-null  int64  
 3   person_home_ownership           21997 non-null  int64  
 4   loan_amnt                       21997 non-null  float64
 5   loan_intent                     21997 non-null  int64  
 6   loan_int_rate                   21997 non-null  float64
 7   cb_person_cred_hist_length      21997 non-null  float64
 8   credit_score                    21997 non-null  int64  
 9   previous_loan_defaults_on_file  21997 non-null  int64  
 10  loan_status                     21997 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 2.0 MB


In [14]:
X=df.iloc[:,:-1]
y=df['loan_status']

In [15]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.3,random_state=42)


In [16]:
model=RandomForestClassifier(n_estimators=200,max_depth=5)
model.fit(Xtrain,ytrain)

,n_estimators,200
,criterion,'gini'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
ypred=model.predict(Xtest)

In [18]:
accuracy_score(ytest,ypred)

0.9445454545454546

In [19]:
with open('rfmodel.pkl','wb')as file:
    pickle.dump(model,file)

In [20]:
import sklearn
print(sklearn.__version__)

1.7.2
